# Testes Automatizados do Pipeline de Dados

Este notebook valida as principais regras de negócio do pipeline de integração de dados meteorológicos e de vazão para previsão mensal de vazão. São utilizados subsets dos arquivos gerados para garantir performance e clareza dos testes.

## 1. Importação das Bibliotecas Necessárias

Vamos importar as bibliotecas essenciais para manipulação e análise dos dados.

In [ ]:
import pandas as pd
import json
import matplotlib.pyplot as plt
from pathlib import Path

## 2. Carregar Subsets dos Arquivos de Dados

Vamos carregar amostras dos arquivos JSON gerados pelo pipeline para validação dos testes.

In [ ]:
# Função utilitária para carregar um subset

def load_subset_json(path, n=1000):
    with open(path) as f:
        data = json.load(f)
    return pd.DataFrame(data[:n])

# Carregar subsets
meteo_subbasin = load_subset_json('../data/meteo_station_subbasin_mapping.json', 1000)
vazao_mensal = load_subset_json('../data/vazao_mensal.json', 1000)
meteo_vazao_joined = load_subset_json('../data/meteo_vazao_joined.json', 1000)
meteo_vazao_shifted = load_subset_json('../data/meteo_vazao_shifted.json', 1000)

# Visualizar exemplos
meteo_subbasin.head(), vazao_mensal.head(), meteo_vazao_joined.head(), meteo_vazao_shifted.head()

## 3. Testes de Regras de Negócio

### 3.1 Join Espacial: Estação Meteorológica x Subbacia
Verifica se todas as estações meteorológicas possuem subbacia associada.

In [ ]:
# Teste: todas as estações têm subbacia associada?
assert meteo_subbasin['Subbasin'].notnull().all(), 'Existem estações meteorológicas sem subbacia associada!'
print('Join espacial: todas as estações possuem subbacia associada. OK!')

### 3.2 Agregação Mensal de Vazão
Verifica se não há duplicidade de (station_id, year, month) e se os valores são médios.

In [ ]:
# Teste: não deve haver duplicidade de (station_id, year, month)
duplicados = vazao_mensal.duplicated(subset=['station_id', 'year', 'month']).sum()
assert duplicados == 0, f'Existem {duplicados} registros duplicados de vazão mensal!'
print('Agregação mensal: não há duplicidade de (station_id, year, month). OK!')

### 3.3 Join Temporal: Meteorologia x Vazão
Verifica se as chaves de junção (ano, mês, subbacia) existem em ambos os datasets.

In [ ]:
# Teste: todas as chaves de meteo_vazao_joined existem em ambos os datasets
chaves_meteo = set(zip(meteo_vazao_joined['year'], meteo_vazao_joined['month'], meteo_vazao_joined['subbasin_id']))
chaves_meteo_orig = set(zip(meteo_subbasin['year'], meteo_subbasin['month'], meteo_subbasin['Subbasin']))
chaves_vazao = set(zip(vazao_mensal['year'], vazao_mensal['month'], vazao_mensal['subbasin_id']))

assert chaves_meteo.issubset(chaves_meteo_orig), 'Existem chaves em meteo_vazao_joined que não estão nos dados meteorológicos!'
assert chaves_meteo.issubset(chaves_vazao), 'Existem chaves em meteo_vazao_joined que não estão nos dados de vazão!'
print('Join temporal: todas as chaves presentes nos datasets originais. OK!')

### 3.4 Deslocamento Temporal e Filtragem
Verifica se o deslocamento foi aplicado corretamente e se não há vazões nulas ou negativas.

In [ ]:
# Teste: não deve haver vazão nula ou negativa
total = len(meteo_vazao_shifted)
na = meteo_vazao_shifted['flow_next_month'].isna().sum()
neg = (meteo_vazao_shifted['flow_next_month'] <= 0).sum()
assert na == 0, f'Existem {na} registros com vazão nula.'
assert neg == 0, f'Existem {neg} registros com vazão <= 0.'
print(f'Deslocamento temporal e filtragem: {total} registros válidos. OK!')

### 3.5 Visualização de Exemplos
Exibe exemplos de registros válidos do dataset final para inspeção manual.

In [ ]:
# Exibir exemplos do dataset final
meteo_vazao_shifted.sample(5, random_state=42)